<a href="https://colab.research.google.com/github/Bjorn455215/B103021055/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q line-bot-sdk #-q 不顯示太多文字

In [ ]:
!pip install -q flask pyngrok flask_ngrok

In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTOTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage, #貼圖套件
    ImageMessage, #圖片套件
    VideoMessage, #影片套件
    LocationMessage #位置套件
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('Line_channel_accesstoken'))
handler = WebhookHandler(userdata.get('Line_channel_token'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai #設置 Google Gemini 模型
#配置 Google Gemini 模型，並定義了一個 ask_gemini 函數，用於向模型提問并获取答案。
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')
def ask_gemini(question):
  reponse = model.generate_content(question)
  return reponse.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'sticker':
           reply = StickerMessage(
            package_id='6362',
            sticker_id='11087924'
           )
        elif action == 'image':
           reply = ImageMessage(
            original_content_url='https://cdn.pixabay.com/photo/2018/01/15/11/36/shiba-inu-3083761_1280.jpg',
            preview_image_url='https://cdn.pixabay.com/photo/2018/01/15/11/36/shiba-inu-3083761_1280.jpg'
           )
        elif action == 'video':
           reply = VideoMessage(
            original_content_url='https://cdn.pixabay.com/video/2024/03/26/205717-927672854_tiny.mp4',
            preview_image_url='https://img.youtube.com/vi/QmMUM3enIEQ/sddefault.jpg'
           )
        elif action == 'location':
           reply = LocationMessage(
            title="國立政治大學附屬高級中學",
            address="116台北市文山區政大一街353號",
            latitude=24.987933167396452,
            longitude=121.5851952617447
           )
        else:
           response = ask_gemini(action)
           reply = TextMessage(text=response)

        line_bot_api.reply_message(
         ReplyMessageRequest(
           reply_token=event.reply_token,
             messages=[
               reply
             ]
         ))



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()